# Импорт библиотек

In [10]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.chrome.service import Service as ChromeService
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup as bs

from time import sleep
import random


pd.set_option('display.max_columns', None)


# Обоснование выбора компании и датасета

[Ссылка на данные](https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers) 



Легенда       
Мы команда продуктовых аналитиков направления кредитные карты в компании American Express.     
В последние данные прослеживается тренд снижения популярности данного продукта.     

В связи с этим необходимо: 
1. Проанализировать отток клиентов банка (отказы от кредитных карт) 
2. Составление профилей пользователь кредитных карт и потенциальных отказников
3. Построение модели, которая бы предсказывала, кто из клиентов наиболее вероятно откажется от продукта 

In [7]:
df = pd.read_csv('BankChurners.csv')


df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis=1)
print(df.shape)
df.head()

(10127, 21)


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [3]:
df.columns

Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
      dtype='object')

In [46]:
df.Income_Category.unique()

array(['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
       '$120K +', 'Unknown'], dtype=object)

In [49]:
df.groupby(['Card_Category'])['CLIENTNUM'].agg('count')

Card_Category
Blue        9436
Gold         116
Platinum      20
Silver       555
Name: CLIENTNUM, dtype: int64

In [47]:
df[df['Income_Category']=='Less than $40K'].groupby(['Card_Category'])['CLIENTNUM'].agg('count')

Card_Category
Blue        3403
Gold          24
Platinum       4
Silver       130
Name: CLIENTNUM, dtype: int64

In [48]:
df[df['Income_Category']=='$120K +'].groupby(['Card_Category'])['CLIENTNUM'].agg('count')

Card_Category
Blue        645
Gold         18
Platinum      4
Silver       60
Name: CLIENTNUM, dtype: int64

# Дополнительные фичи

# Дополнение датафрейма
1. Цена обслуживания разных типов карт (парсинг)   
Источник: [American Express](https://www.americanexpress.com/us/credit-cards/?category=all&eep=25330&intlink=US-Axp-Shop-Consumer-VAC-Prospect-all-StickyFilter)
2. Категории населения по возрасту + статистика в разрезе США (файл)  
Источник: [Neilsberg](https://www.neilsberg.com/insights/united-states-population-by-age/)

In [ ]:
def site_open_func(site: str, browser: str = 'firefox'):
    ua = UserAgent()

    if browser == 'firefox':
        service = FirefoxService(GeckoDriverManager().install())
        user_agent = ua.firefox
        options = FirefoxOptions()
        options.set_preference("general.useragent.override", user_agent)

        driver = webdriver.Firefox(service=service, options=options)
    else:
        raise ValueError("Unsupported browser. Choose 'firefox' or 'chrome'.")

    url = site
    driver.get(url)
    sleep(3)

    return driver, url

In [ ]:
# Флаг для парсинга цен по картам
parse_card_info_flg = False

In [ ]:
if parse_card_info_flg:
    site = 'https://www.americanexpress.com/us/credit-cards/?category=all&eep=25330&intlink=US-Axp-Shop-Consumer-VAC-Prospect-all-StickyFilter'
    driver = site_open_func(site, 'firefox')
    page_blue_card = bs(driver.page_source)
    df_card_info = pd.DataFrame(columns=['Card', 'Annual fee'])

    for card in page_blue_card.find_all(True):
        try:
            if card.find('strong', class_='dls-deep-blue', string='Annual Fee'):
                card_name = card.find_previous('h2').get_text(strip=True)
                fee_span = card.find('span')
                html_string = str(fee_span)

                import re
                dollar_amounts = re.findall(r'\$\d+', html_string)
                
                if dollar_amounts:
                    annual_fee = dollar_amounts[0].replace('$', '')
                    # print(card_name, annual_fee)

                    card_exists = (df_card_info['Card'] == card_name).any()
    
                    if annual_fee is not None and not card_exists:
                        df_card_info.loc[len(df_card_info)] = {
                            'Card': card_name,
                            'Annual fee': annual_fee
                        }
                    
        except Exception as e:
            pass

In [ ]:
if parse_card_info_flg:

    df_card_info.loc[len(df_card_info), :] = ['Blue Cash Everyday Card', 0]
    df_card_info['Annual fee'] = df_card_info['Annual fee'].astype(int)
    df_card_info['Card'] = df_card_info['Card'].str.replace('®', ' ').str.strip()
    df_card_info.loc[2, 'Annual fee'] = 95 #точечная корректировка (комплексное условие)

    dict_card_names = {
                    'Blue':'Blue Cash Everyday Card', 
                    'Silver':'American Express Green Card', 
                    'Gold':'American Express Gold Card', 
                    'Platinum':'The Platinum Card', 
                    }
    dict_card_names_reverse = {v: k for k, v in dict_card_names.items()}


    df_card_info.loc[:, 'Card_Category'] = df_card_info['Card'].map(dict_card_names_reverse)
    df_card_info.Card_Category = df_card_info.Card_Category.fillna('Other')
    df_card_info.to_csv('Парсинг_цена_карт.csv', index=False)

In [ ]:
df_age_group = pd.read_csv('population_age_category.csv')
df_age_group.Population = df_age_group.Population.str.replace(',', '').astype('float')
df_age_group['% of Total Population'] = df_age_group['% of Total Population'].str.replace('%', '').astype('float') / 100

print(df_age_group.shape)
df_age_group.head(2)

(18, 3)


,Age Group,Population,% of Total Population
0,Under 5 years,18939899.0,0.0570
1,5 to 9 years,20102351.0,0.0605


In [215]:
def extract_age_bounds(age_str):
    numbers = re.findall(r'\d+', age_str)
    
    if "Under" in age_str:
        return 0, int(numbers[0]) - 1
    elif "and over" in age_str:
        return int(numbers[0]), None
    elif len(numbers) == 2:
        return int(numbers[0]), int(numbers[1])
    else:
        return None, None

df_age_group[['age_from', 'age_to']] = df_age_group['Age Group'].apply(
    lambda x: pd.Series(extract_age_bounds(x))
)

In [216]:
# Присоединяем новые фичи
df_2 = df.copy().reset_index(drop=True)
df_age_group = df_age_group.reset_index(drop=True)


def find_age_group(age):
    mask = (
        (df_age_group['age_from'] <= age) & (df_age_group['age_to'] >= age)
    )
    matching_rows = df_age_group[mask]
    
    if not matching_rows.empty:
        return matching_rows.iloc[0][['Age Group', 'Population', '% of Total Population']]
    else:
        # Возвращаем NaN или дефолтные значения, если группа не найдена
        return pd.Series([None, None, None], index=['Age Group', 'Population', '% of Total Population'])

# Применяем и создаем новые колонки
age_group_data = df_2['Customer_Age'].apply(find_age_group)
df_2 = pd.concat([df_2, age_group_data.apply(pd.Series)], axis=1)

df_2.head(2)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Age Group,Population,% of Total Population
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,45 to 49 years,20051874.0,0.0603
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,45 to 49 years,20051874.0,0.0603


In [221]:
df_card_info = pd.read_csv('Парсинг_цена_карт.csv')
df_2 = df_2.merge(df_card_info, how='left', on='Card_Category')

In [224]:
df_2.to_parquet('BankChurners_add_features.parquet')